In [1]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is NOT available")

GPU is NOT available


In [7]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
 # Example classifier

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from evaluator import *

'''import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau'''



'import tensorflow as tf\nfrom tensorflow.keras.layers import Input, Dense, Dropout\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.optimizers import Adam\nfrom tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy\nfrom tensorflow.keras.callbacks import ReduceLROnPlateau'

In [8]:
# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test_true = dat['X_test']
S_test_true = dat['S_test']

print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)

# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

# Split the data (final _ to keep split data untouched and be able to reload in file)
# X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_ = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# Refresh training data
# X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [21]:
# Keep the original index for later use
X['original_index'] = X.index

# Step 1: Keep original index by resetting it if not already in a suitable format
X.reset_index(drop=True, inplace=True)
Y = Y.reset_index(drop=True)
S = S.reset_index(drop=True)

# Create a combined Y and S DataFrame for stratification
YS = pd.DataFrame({'Y': Y, 'S': S})

# Split the data while stratifying based on Y and S
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=0, stratify=YS)
print('train',X_train.shape,Y_train.shape,S_train.shape)
print('test',X_test.shape,Y_test.shape,S_test.shape)

# Create XS and XnotS subsets for training and test sets
XS_train = X_train[S == True]
YS_train = Y_train[S == True]

XnotS_train = X_train[S == False]
YnotS_train = Y_train[S == False]

XS_test = X_test[S == True]
YS_test = Y_test[S == True]

XnotS_test = X_test[S == False ]
XnotS_test = X_test[S == False ]

XS_train.head(5)

train (19424, 769) (19424,) (19424,)
test (8325, 769) (8325,) (8325,)


/tmp/ipykernel_102235/2873319695.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  XS_train = X_train[S == True]
/tmp/ipykernel_102235/2873319695.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  XnotS_train = X_train[S == False]
/tmp/ipykernel_102235/2873319695.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  XS_test = X_test[S == True]
/tmp/ipykernel_102235/2873319695.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  XnotS_test = X_test[S == False ]


,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,original_index
4002,-0.352127,-0.011900,0.464177,-0.429489,0.022461,-0.355041,0.404765,0.256324,-0.171173,-0.386853,...,-0.659120,-0.018598,-0.461626,0.263559,0.106523,-0.426319,-0.123851,0.176851,0.700239,4002
18293,-0.035066,-0.298860,-0.479104,-0.626332,-0.019420,0.021350,0.052250,-0.043124,-0.156429,-0.086376,...,-0.595587,-0.015968,-0.765048,0.323093,0.170880,-0.186892,0.661593,0.406664,0.397004,18293
15870,0.059748,-0.641063,-0.144781,0.100073,0.337262,-0.558625,0.539921,0.473667,-0.363462,-0.405527,...,-0.235132,-0.101076,-0.915928,0.364703,0.321795,-0.059579,-0.066653,0.360908,0.337702,15870
6207,-0.424646,-0.094270,-0.975104,-0.402996,-0.248395,0.203762,0.170925,0.119010,-0.242348,-0.148573,...,-0.755281,-0.236163,-0.255455,-0.083623,-0.183295,-0.180494,0.264272,0.158392,0.278496,6207
25494,-0.144659,-0.019726,-0.872632,-0.925582,0.001781,0.324370,-0.124459,0.284658,-0.412551,0.303603,...,0.136539,-0.418398,-1.001702,-0.187408,-0.229588,-0.343264,0.321463,0.196229,0.627135,25494


In [22]:
print(Y_train[S == True].shape)
Y_train[S == True].head()

(8953,)


4002     13
18293    19
15870    24
6207     21
25494    18
dtype: int64

In [18]:
# Prepare Logistic Regression models
model_X = LogisticRegression(random_state=0, max_iter=5000)
model_XS = LogisticRegression(random_state=0, max_iter=5000)
model_XnotS = LogisticRegression(random_state=0, max_iter=5000)

# Train the models
model_X.fit(X_train, Y_train)
model_XS.fit(XS_train, Y_train[S==1])
model_XnotS.fit(XnotS_train, Y_train[S==0])


TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:

# Predictions on test and train data
Y_pred = model_X.predict(X_test.drop(columns=['S', 'original_index']))
YS_pred = model_XS.predict(XS_test.drop(columns=['S', 'original_index']))
YnotS_pred = model_XnotS.predict(XnotS_test.drop(columns=['S', 'original_index']))

Y_pred_train = model_X.predict(X_train.drop(columns=['S', 'original_index']))
YS_pred_train = model_XS.predict(XS_train.drop(columns=['S', 'original_index']))
YnotS_pred_train = model_XnotS.predict(XnotS_train.drop(columns=['S', 'original_index']))

# Calculate F1 scores
f1_score_X_test = f1_score(Y_test, Y_pred, average='macro')
f1_score_XS_test = f1_score(Y_test[XS_test.index], YS_pred, average='macro')
f1_score_XnotS_test = f1_score(Y_test[XnotS_test.index], YnotS_pred, average='macro')

f1_score_X_train = f1_score(Y_train, Y_pred_train, average='macro')
f1_score_XS_train = f1_score(Y_train[XS_train.index], YS_pred_train, average='macro')
f1_score_XnotS_train = f1_score(Y_train[XnotS_train.index], YnotS_pred_train, average='macro')

# Output F1 scores
print("F1 Scores for Test Data:")
print(f"All Data: {f1_score_X_test}, S Data: {f1_score_XS_test}, Not S Data: {f1_score_XnotS_test}")
print("F1 Scores for Train Data:")
print(f"All Data: {f1_score_X_train}, S Data: {f1_score_XS_train}, Not S Data: {f1_score_XnotS_train}")

# Reconstructing the final DataFrame with predictions
# Note: For the final_predictions DataFrame, additional manipulation is needed.
# Since the structure and desired output were not fully specified in the requirements, below is a conceptual approach.

# Merge predictions back based on original indices to maintain order
# This step is conceptual and may require adjustments based on actual DataFrame structures and desired output format.
test_predictions = pd.DataFrame({
    'Y_test': Y_test,
    'S_test': S_test,
    'Y_pred': Y_pred,
    'Y_pred_train': np.nan,  # Placeholder, actual matching by index needed
    'YS_pred': np.nan,  # Placeholder for XS_test predictions, actual matching by index needed
    'YnotS_pred': np.nan,  # Placeholder for XnotS_test predictions, actual matching by index needed
    'YS_pred_train': np.nan,  # Placeholder, actual matching by index needed
    'YnotS_pred_train': np.nan  # Placeholder, actual matching by index needed
}, index=indices_test)



In [6]:
XF=X[S==1]
XH=X[S==0]
print(XF.shape,XH.shape)

YF=Y[S==1]
YH=Y[S==0]
print(YF.shape,YH.shape)

XF.head(5)

(12793, 768) (14956, 768)
(12793,) (14956,)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
6286,0.056359,0.003188,-0.726052,-0.616777,0.064091,-0.068786,0.384864,0.363644,-0.285173,-0.430806,...,-0.049452,-0.587354,-0.004443,-0.924459,-0.402672,0.170605,-0.074348,0.130570,0.328335,0.139750
12083,-0.565799,0.118481,0.185003,-0.692792,-0.056820,0.039130,0.241766,-0.113560,-0.138898,-0.768206,...,-0.071413,-0.310701,0.157026,-0.691291,-0.273562,0.121746,-0.394475,-0.463161,0.132829,0.264848
12702,-0.684493,-0.188508,-0.670183,-0.572966,-0.117222,0.200948,0.304471,0.259940,-0.248586,-0.271820,...,-0.236687,-0.772385,-0.041944,-0.674941,0.239194,0.054098,-0.203181,0.663840,0.376015,0.500834
17728,-0.442187,-0.120117,0.321693,-0.552460,-0.083063,-0.383489,0.259511,0.199604,-0.227831,-0.330898,...,0.101770,-0.598164,-0.170784,-0.412672,0.253084,0.072861,-0.407497,-0.076331,0.210154,0.622336
15415,0.467783,-0.013451,-0.532544,-0.704010,-0.145949,-0.195713,0.415697,-0.249390,-0.451609,0.296516,...,-0.046805,-0.262006,0.052341,-1.049271,-0.142108,-0.244743,-0.421750,0.151491,0.070547,0.399572


Epoch 1/10
243/243 [==============================] - 3s 8ms/step - loss: 2.2830 - main_output_loss: 2.2833 - adversary_output_loss: 3.1513 - main_output_accuracy: 0.4115 - adversary_output_auc: 0.4724 - val_loss: 1.6796 - val_main_output_loss: 1.6804 - val_adversary_output_loss: 7.5507 - val_main_output_accuracy: 0.5591 - val_adversary_output_auc: 0.5000 - lr: 1.0000e-04
Epoch 2/10
243/243 [==============================] - 2s 7ms/step - loss: 1.6250 - main_output_loss: 1.6265 - adversary_output_loss: 15.6673 - main_output_accuracy: 0.5599 - adversary_output_auc: 0.5000 - val_loss: 1.3585 - val_main_output_loss: 1.3612 - val_adversary_output_loss: 26.4842 - val_main_output_accuracy: 0.6147 - val_adversary_output_auc: 0.5000 - lr: 1.0000e-04
Epoch 3/10
243/243 [==============================] - 1s 6ms/step - loss: 1.3886 - main_output_loss: 1.3927 - adversary_output_loss: 40.6770 - main_output_accuracy: 0.6095 - adversary_output_auc: 0.5000 - val_loss: 1.1891 - val_main_output_loss: 1.

In [33]:
# Evaluate model on test set
evaluation = model.evaluate(X_test, {'main_output': Y_test, 'adversary_output': S_test})


261/261 [==============================] - 1s 3ms/step - loss: 0.0360 - main_output_loss: 0.8787 - adversary_output_loss: 1685.4054 - main_output_accuracy: 0.7356 - adversary_output_auc: 0.5000


In [34]:
# Extract shared representation model
representation_model = Model(inputs=model.input, outputs=model.get_layer('shared_representation').output)

# Transform X_train and X_test
X_train_transformed_2 = representation_model.predict(X_train)
X_test_transformed_2 = representation_model.predict(X_test)

261/261 [==============================] - 1s 2ms/step


In [35]:
# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(max_iter=5000)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

Accuracy on transformed test data: 0.7577177177177177

final 0.7319076217263948
macro_fscore 0.6526436809324047
1-eval_scores 0.8111715625203849


In [39]:
classwise_loss(Y_test, Y_pred_2)
classwise_accuracy(Y_test, Y_pred_2)

2024-03-06 22:06:17.056611: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (0 for input with 0 dimension(s)


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid reduction dimension (0 for input with 0 dimension(s) [Op:Mean] name: 

In [38]:
import tensorflow as tf

def classwise_accuracy(y_true, y_pred):
    """
    Custom metric function that calculates the accuracy for each class individually
    and then averages these accuracies.
    
    Args:
        y_true: Tensor of true labels, one-hot encoded.
        y_pred: Tensor of predicted labels, as probabilities.

    Returns:
        A scalar tensor representing the average class-wise accuracy.
    """
    # Convert probabilities to predicted class (highest probability)
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_classes = tf.argmax(y_true, axis=-1)
    
    # Calculate accuracy for each class
    class_accuracies = tf.cast(tf.equal(y_true_classes, y_pred_classes), tf.float32)
    
    # Reduce across all dimensions but the first (batch dimension)
    # to get accuracy per class, then take the mean across classes
    classwise_acc = tf.reduce_mean(class_accuracies, axis=0)
    overall_acc = tf.reduce_mean(classwise_acc)
    
    return overall_acc

# Example usage with a model compilation in TensorFlow/Keras
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[classwise_accuracy])
